<b>Create Sidebar Navigation - best viewed in full screen</b>

In [1]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

<IPython.core.display.Javascript object>

# Set up SQLite

In [2]:
# Import dependencies
import sqlite3

In [3]:
## Define connection and cursor 
conn = sqlite3.connect("../sqlite/wake_housing_database.db")
c = conn.cursor()

## Create Tables

In [4]:
# Erase previous data (start with a clean slate)
c.execute("DROP TABLE IF EXISTS housing;")
c.execute("DROP TABLE IF EXISTS crime;")
c.execute("DROP TABLE IF EXISTS noise;")

In [5]:
# Verify there are no tables
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('combined_data',)]


In [6]:
# Create tables 
# Housing table
c.execute("""CREATE TABLE housing (
               Total_Value INTEGER NOT NULL,
               Address VARCHAR(100) PRIMARY KEY NOT NULL,
               City CHAR(50) NOT NULL,
               Zip_Code INTEGER NOT NULL,
               Planning_Jurisdiction INTEGER NOT NULL,
               Zoning VARCHAR(50) NOT NULL,
               Acreage FLOAT NOT NULL,
               Sqft INTEGER NOT NULL,
               Age INTEGER NOT NULL,
               Bath FLOAT NOT NULL,
               Remodel_Addition INTEGER NOT NULL,
               Style INTEGER NOT NULL,
               Latitude FLOAT NOT NULL,
               Longitude FLOAT NOT NULL
               )""")

# Crime table
c.execute("""CREATE TABLE crime (
               City CHAR(50) PRIMARY KEY NOT NULL,
               Personal_Crime_Count INTEGER NOT NULL,
               Personal_Name_Count INTEGER NOT NULL,
               Property_Crime_Count INTEGER NOT NULL,
               Property_Name_Count INTEGER NOT NULL,
               Drug_Crime_Count INTEGER NOT NULL,
               Drug_Name_Count INTEGER NOT NULL,
               Total_Crime_Count INTEGER NOT NULL,
               Total_Name_Count INTEGER NOT NULL,
               FOREIGN KEY(City) REFERENCES housing(City)
               )""")

# Noise table
c.execute("""CREATE TABLE noise (
               Address VARCHAR(100) PRIMARY KEY NOT NULL,
               City CHAR(50) NOT NULL,
               Zip_Code INTEGER NOT NULL,
               Latitude FLOAT NOT NULL,
               Longitude FLOAT NOT NULL,
               Decibel FLOAT NOT NULL,
               Decibel_Scale INTEGER NOT NULL
               )""")

# Add tables

In [7]:
# Import dependencies
import pandas as pd

In [8]:
# Create dataframes
housing_df = pd.read_csv("../resources/clean/housing_data_with_coordinates.csv")
crime_df = pd.read_csv("../resources/clean/crime_count_data_clean.csv")
noise_df = pd.read_csv("../resources/clean/noise_data_clean.csv")

In [9]:
# Connect dataframes to SQLite database
housing_df.to_sql("housing", conn, if_exists="replace")
crime_df.to_sql("crime", conn, if_exists="replace")
noise_df.to_sql("noise", conn, if_exists="replace")

# Join tables

In [10]:
# Join housing, crime, and noise tables
joined_tables_df = '''
    SELECT *
    FROM housing
    LEFT JOIN crime ON housing.City = crime.City
    LEFT JOIN noise ON housing.Address = noise.Address
'''

all_data_df = pd.read_sql_query(joined_tables_df, conn)

# Show data types of resulting table
all_data_df.dtypes

index                      int64
Total_Value                int64
Address                   object
City                      object
Zip_Code                   int64
Planning_Jurisdiction      int64
Zoning                    object
Acreage                  float64
Sqft                       int64
Age                        int64
Bath                     float64
Remodel_Addition           int64
Style                      int64
Latitude                 float64
Longitude                float64
index                      int64
City                      object
Personal_Crime_Count       int64
Personal_Name_Count        int64
Property_Crime_Count       int64
Property_Name_Count        int64
Drug_Crime_Count           int64
Drug_Name_Count            int64
Total_Crime_Count          int64
Total_Name_Count           int64
index                      int64
Address                   object
City                      object
Zip_Code                   int64
Latitude                 float64
Longitude 

## Clean Joined Table

In [11]:
# Remove duplicated columns
all_data_df = all_data_df.loc[:, ~all_data_df.columns.duplicated()]

# Drop extra index column
all_data_df = all_data_df.drop(columns=["index"])

# Check columns left
all_data_df.dtypes

Total_Value                int64
Address                   object
City                      object
Zip_Code                   int64
Planning_Jurisdiction      int64
Zoning                    object
Acreage                  float64
Sqft                       int64
Age                        int64
Bath                     float64
Remodel_Addition           int64
Style                      int64
Latitude                 float64
Longitude                float64
Personal_Crime_Count       int64
Personal_Name_Count        int64
Property_Crime_Count       int64
Property_Name_Count        int64
Drug_Crime_Count           int64
Drug_Name_Count            int64
Total_Crime_Count          int64
Total_Name_Count           int64
Decibel                  float64
Decibel_Scale              int64
dtype: object

## Add table to SQLite Database

In [12]:
# Make sure there is no table to start with
c.execute("DROP TABLE IF EXISTS combined_data;")

# Add table to sqlite database
c.execute("""CREATE TABLE combined_data (
               Total_Value INTEGER NOT NULL,
               Address VARCHAR(100) PRIMARY KEY NOT NULL,
               City CHAR(50) NOT NULL,
               Zip_Code INTEGER NOT NULL,
               Planning_Jurisdiction INTEGER NOT NULL,
               Zoning VARCHAR(50) NOT NULL,
               Acreage FLOAT NOT NULL,
               Sqft INTEGER NOT NULL,
               Age INTEGER NOT NULL,
               Bath FLOAT NOT NULL,
               Remodel_Addition INTEGER NOT NULL,
               Style INTEGER NOT NULL,
               Latitude FLOAT NOT NULL,
               Longitude FLOAT NOT NULL,
               Personal_Crime_Count INTEGER NOT NULL,
               Personal_Name_Count INTEGER NOT NULL,
               Property_Crime_Count INTEGER NOT NULL,
               Property_Name_Count INTEGER NOT NULL,
               Drug_Crime_Count INTEGER NOT NULL,
               Drug_Name_Count INTEGER NOT NULL,
               Total_Crime_Count INTEGER NOT NULL,
               Total_Name_Count INTEGER NOT NULL,
               Decibel FLOAT NOT NULL,
               Decibel_Scale INTEGER NOT NULL
               )""")

# Insert data
all_data_df.to_sql("combined_data", conn, if_exists="replace")

In [13]:
# Verify list of tables
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('housing',), ('crime',), ('noise',), ('combined_data',)]


# Close SQLite connection and show data

In [14]:
#close connection
conn.close

<function Connection.close>

In [15]:
# Show data in database
print(all_data_df)

        Total_Value              Address        City  Zip_Code  \
0            249321       2457 BERTIE DR     RALEIGH     27610   
1            159933   2848 PROVIDENCE RD     RALEIGH     27610   
2            222624    409 S LAKESIDE DR     RALEIGH     27606   
3            150723     540 MARSHBURN RD     WENDELL     27591   
4            140801      605 WOODLAND RD     RALEIGH     27603   
...             ...                  ...         ...       ...   
241763       474932          500 KENT DR        CARY     27511   
241764       235612    1305 PONDEROSA DR     RALEIGH     27603   
241765       897806   1809 STILLWATER DR     RALEIGH     27607   
241766        98296  7081 FORESTVILLE RD  KNIGHTDALE     27545   
241767       552648      110 SHEPHERD ST     RALEIGH     27607   

        Planning_Jurisdiction Zoning  Acreage  Sqft  Age  Bath  ...  \
0                           9    R-4     0.21  1828   59   2.0  ...   
1                           9    R-4     0.46  1240   53   1.0  .